In [ ]:
!pip install transformers torch
!pip install transformers sentencepiece
!pip install datasets


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset


# dataset = load_dataset("maiurilorenzo/divina-commedia")
dataset = load_dataset("giambono/commedia_en")

# Display the first few entries
df = dataset["train"].to_pandas()

README.md:   0%|          | 0.00/474 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/493k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14233 [00:00<?, ? examples/s]

In [26]:
df_select = df[(df['volume'] == "Inferno")]

In [27]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def process_verses(df, text_column='text', volume_column='volume', canto_column='canto',
                    tercet_column='tercet', verse_number_column='verse_number'):
    """
    Processes each verse from the input DataFrame, generating five paraphrased versions for each verse
    using a transformer model and storing them in a new DataFrame along with the original metadata.

    Parameters:
        df (pd.DataFrame): Input DataFrame containing the verses of the Divine Comedy.
        text_column (str): Column name for the verse text.
        volume_column (str): Column name for the volume (Inferno, Purgatorio, Paradiso).
        canto_column (str): Column name for the canto number.
        tercet_column (str): Column name for the tercet number.
        verse_number_column (str): Column name for the verse number.

    Returns:
        pd.DataFrame: A new DataFrame containing the original metadata and five paraphrased outputs for each verse.
    """
    tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
    model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    output_data = []

    for idx_verse, row in df.iterrows():
        print(row.canto)
        verse_text = row[text_column]
        text = "paraphrase: " + verse_text + " </s>"
        encoding = tokenizer.encode_plus(text, pad_to_max_length=True, return_tensors="pt")
        input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

        outputs = model.generate(
            input_ids=input_ids, attention_mask=attention_masks,
            max_length=256,
            do_sample=True,
            top_k=200,
            top_p=0.95,
            early_stopping=True,
            num_return_sequences=5
        )

        paraphrased_texts = [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in outputs]

        for i, paraphrased_text in enumerate(paraphrased_texts, 1):
            output_data.append({
                'volume': row[volume_column],
                'canto': row[canto_column],
                'tercet': row[tercet_column],
                'verse_number': row[verse_number_column],
                'original_text': verse_text,
                'output_number': i,
                'transformed_text': paraphrased_text,
                'expected_index': idx_verse
            })

    return pd.DataFrame(output_data)

# Example usage:
# df = pd.read_csv('divine_comedy.csv')  # Assuming a CSV file with the verses
# result_df = process_verses(df)
# display(result_df)


In [28]:
result_df = process_verses(df_select)


Canto I


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I
Canto I


In [29]:
result_df.to_csv('paraphrased_verses.csv', index=False)

In [30]:
result_df.to_parquet('paraphrased_verses.parquet', index=False)

In [31]:
result_df.head()

,volume,canto,tercet,verse_number,original_text,output_number,transformed_text
0,Inferno,Canto I,1,1,Midway upon the journey of our life,1,Midway on the journey of our life
1,Inferno,Canto I,1,1,Midway upon the journey of our life,2,Midway along the journey of our life
2,Inferno,Canto I,1,1,Midway upon the journey of our life,3,Our path through our life takes in
3,Inferno,Canto I,1,1,Midway upon the journey of our life,4,Midway on the journey of our life.
4,Inferno,Canto I,1,1,Midway upon the journey of our life,5,Midway through the journey of our life
